# Install VideoImageTools

To install VideoImageTools go to next link https://github.com/trucomanx/VideoImageTools

# Install OpenPifPafTools

To install OpenPifPafTools go to next link https://github.com/trucomanx/OpenPifPafTools


In [81]:
import platform, sys, os

# Variables

In [82]:
## Model of network
#model_type = 'efficientnet_b3'
model_type = 'mobilenet_v3';
#model_type = 'inception_v3';
#model_type = 'inception_resnet_v2';
#model_type = 'resnet_v2_50';

input_mp4_file='drhouse_mini_cut.mp4';#'2023-03-20-172540_mini.mp4';#drhouse_mini_cut.mp4

# Endereços

In [83]:
## input
if platform.system()=='Linux':
    vin_dir_path ='/mnt/boveda/DATASETs/PATIENT-VIDEOS/dataset-toy';
else:
    vin_dir_path ='C:\\Dados\\Fernando\\DATASET\\PATIENT-VIDEOS\\dataset-toy';

## output
if platform.system()=='Linux':
    vout_dir_path='/mnt/boveda/DOCTORADO2/cnn_face_emotion/test_over_video';
else:
    vout_dir_path='C:\\Dados\\Fernando\\OUTPUTS\\cnn_face_emotion\\test_over_video';

# Biblioteca local

In [84]:
sys.path.append('library');

In [85]:
import Classifier as mylib

# Bibliotecas externas

In [86]:
import VideoImageTools as vit
import OpenPifPafTools.OpenPifPafAnnotations as opp
import OpenPifPafTools.OpenPifPafGetData as oppgd


import cv2
import openpifpaf
from PIL import Image



# If command line

In [87]:
for n in range(len(sys.argv)):
    if sys.argv[n]=='--model':
        model_type=sys.argv[n+1];

print('model_type:',model_type)

model_type: mobilenet_v3


In [88]:
vin_path =os.path.join(vin_dir_path ,input_mp4_file);
vout_path=os.path.join(vout_dir_path,os.path.splitext(os.path.basename(input_mp4_file))[0]+'_'+model_type+'.mp4');


# Creating output directory

In [89]:
try:
    os.makedirs(vout_dir_path) 
    print('      Created output_dir::',vout_dir_path)
except: 
    print('Pass creating output_dir::',vout_dir_path)
    pass


Pass creating output_dir:: /mnt/boveda/DOCTORADO2/cnn_face_emotion/test_over_video


# Auxiliar functions

In [90]:


def my_func(PredT,frame):
    Clf=PredT[0];
    predictor=PredT[1];

    categories=['angry','disgusted','fearful','happy','neutral','sad','surprised'];
    
    img_tmp = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB);
    pil_im = Image.fromarray(img_tmp);
    
    height, width, channels = frame.shape
    
    annotation, gt_anns, image_meta = predictor.pil_image(pil_im);
    
    for annot in annotation: 
        (xi,yi,xo,yo)=oppgd.get_face_bounding_rectangle(annot.data,factor=1.0);
        xi=int(xi);        yi=int(yi);
        xo=int(xo);        yo=int(yo);
        
        if xo<0:
            xo=0;
        if xo>=width:
            xo=width-1;
        if yo<0:
            yo=0;
        if yo>=height:
            yo=height-1;
        
        if xi!=xo and yi!=yo:
            color=(0,255,0);
            thickness=2;

            pil_crop=pil_im.crop((xi,yi,xo,yo));

            res=Clf.is_pil_patient(pil_crop);

            texto=categories[res];

            frame = cv2.putText(  frame,
                                  texto,
                                  org = (int(xi), int((yi+yo)/2)),
                                  fontFace = cv2.FONT_HERSHEY_DUPLEX,
                                  fontScale = 2.0,
                                  color = (255, 0, 0),
                                  thickness = thickness
                                )

            cv2.rectangle(frame,(xi,yi),(xo,yo),color,thickness);
    return frame;




In [91]:
    
Clf=mylib.FaceEmotionClassifier(model_type);
predictor = openpifpaf.Predictor(checkpoint='shufflenetv2k16')#'shufflenetv2k16-wholebody'
vit.apply_func_predictor_over_video_outmp4(my_func,(Clf,predictor),vin_path,vout_path)

print('working end')

Loading model: mobilenet_v3
Model mobilenet_v3 loaded.
Loaded layer with mobilenet_v3


Working: |           [ time left: 00:00 ]      

working end
